<a href="https://colab.research.google.com/github/alxmarqs/LLMtopics/blob/main/gsi073_aula0_seq2seanalisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação dos dados

Esta tarefa é inverter sequências de caracteres. Exemplo: **aabcd** em **dcbaa**.


In [ ]:
import torch
import torch.nn as nn
import random

chars = list("abcd ")
vocab = {ch: i for i, ch in enumerate(chars)} # Cada letra, ganha um número
inv_vocab = {i: ch for ch, i in vocab.items()}# Tabela de decodificação
vocab_size = len(vocab)

def encode(s): # Codifica letras em números
    return torch.tensor([vocab[c] for c in s], dtype=torch.long)

def decode(t): # Decodifica números em letras
    return ''.join(inv_vocab[int(x)] for x in t)

def random_seq(n=5): # Cria novas sequências
    return ''.join(random.choice(chars[:-1]) for _ in range(n))

# Gerar dados
pairs = [(encode(s), encode(s[::-1])) for s in [random_seq() for _ in range(50000)]]

max_len = max(len(x) for x, _ in pairs) # pega maior sequência

def pad(x):  # Preenche conjunto de dados em pad no último índice
    return torch.cat([x, torch.tensor([vocab[' ']] * (max_len - len(x)))], dim=0)

inputs = torch.stack([pad(x) for x, _ in pairs])
targets = torch.stack([pad(y) for _, y in pairs])

train_ds = torch.utils.data.TensorDataset(inputs, targets)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Veja um par

In [ ]:
print(pairs[1])

# Definição do modelo Seq2Seq com GRU

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        _, h = self.gru(x)
        return h  # [1, B, H]

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        """
        x: tensor que indica a parte prévia correta
        h: tensor que indica o estado do encoder da parte prévia
        """
        x = self.embed(x)
        out, h = self.gru(x, h)
        logits = self.fc(out)
        return logits, h # retorna o estado latente para atualizar o estado

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        h = self.encoder(src)
        # usa contexto correto anterior e estado atual para prever o tgt[:, -1]
        logits, _ = self.decoder(tgt[:, :-1], h)
        return logits

# Código para usar o modelo treinado: inferência

In [ ]:
def decode_step(decoder, token, h):
    logits, h = decoder(token, h) # obtém logits e atualiza estado da sequência
    next_token = logits[:, -1, :].argmax(-1, keepdim=True)
    return next_token, h

def predict(model, seq, max_len=10):
    model.eval()
    with torch.no_grad():
        src = pad(encode(seq)).unsqueeze(0).to(device, dtype=torch.long)
        h = model.encoder(src) # Obtém estado do modelo após processar entrada inicial

        # 'token' representa a geração passo a passo da sequência invertida
        token = torch.tensor([[vocab[' ']]], dtype=torch.long, device=device)
        seq_invertida = []
        for _ in range(max_len):
            token, h = decode_step(model.decoder, token, h)
            seq_invertida.append(token.item())
        return decode(seq_invertida)

# Preparação para treino

In [ ]:
emb_size = 32
hidden_size = 64
encoder = Encoder(vocab_size, emb_size, hidden_size)
decoder = Decoder(vocab_size, emb_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[' ']) # ignora o pad: " "
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

# Execução do treino

In [ ]:
for epoch in range(10):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device, dtype=torch.long), yb.to(device, dtype=torch.long)
        opt.zero_grad()
        logits = model(xb, yb)
        loss = loss_fn(logits.reshape(-1, vocab_size), yb[:, 1:].reshape(-1))
        loss.backward()
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: loss={total_loss/len(train_dl):.4f}")

# Vamos testar

In [ ]:
for _ in range(10):
    s = random_seq()
    pred = predict(model, s, max_len=len(s))
    print(f"{s} -> {pred}")


# Exercício
Compare o resultado do uso do encoder de de sequências muito similares e muito diferentes. Por exemplo, codifique "aaaabb", "bbaaab", "cbcaccc" e "cccacbc" e depois faça uma figura das 2 componentes principais usando o método Principal Components Analysis (PCA) do pacote `sklearn.decomposition.PCA`.

In [ ]:
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np

# Configuração dos dados
chars = list("abcd ")
vocab = {ch: i for i, ch in enumerate(chars)}
inv_vocab = {i: ch for ch, i in vocab.items()}
vocab_size = len(vocab)

def encode(s):
    return torch.tensor([vocab[c] for c in s], dtype=torch.long)

def decode(t):
    return ''.join(inv_vocab[int(x)] for x in t)

def random_seq(n=5):
    return ''.join(random.choice(chars[:-1]) for _ in range(n))

# Gerar dados de treino
pairs = [(encode(s), encode(s[::-1])) for s in [random_seq() for _ in range(50000)]]
max_len = max(len(x) for x, _ in pairs)

def pad(x):
    return torch.cat([x, torch.tensor([vocab[' ']] * (max_len - len(x)))], dim=0)

inputs = torch.stack([pad(x) for x, _ in pairs])
targets = torch.stack([pad(y) for _, y in pairs])

train_ds = torch.utils.data.TensorDataset(inputs, targets)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Definição do modelo
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        _, h = self.gru(x)
        return h

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embed(x)
        out, h = self.gru(x, h)
        logits = self.fc(out)
        return logits, h

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        h = self.encoder(src)
        logits, _ = self.decoder(tgt[:, :-1], h)
        return logits

# Treinar modelo
emb_size = 32
hidden_size = 64
encoder = Encoder(vocab_size, emb_size, hidden_size)
decoder = Decoder(vocab_size, emb_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[' '])
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

print("Treinando modelo...")
for epoch in range(10):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device, dtype=torch.long), yb.to(device, dtype=torch.long)
        opt.zero_grad()
        logits = model(xb, yb)
        loss = loss_fn(logits.reshape(-1, vocab_size), yb[:, 1:].reshape(-1))
        loss.backward()
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: loss={total_loss/len(train_dl):.4f}")

# EXERCÍCIO: Análise com PCA
print("\n" + "="*50)
print("ANÁLISE PCA DAS REPRESENTAÇÕES DO ENCODER")
print("="*50)

# Sequências para análise
sequences = [
    "aaaabb",   # Muitos 'a' seguidos de 'b'
    "bbaaab",   # Começa com 'b', depois 'a', termina com 'b'
    "cbcaccc",  # Mix com muitos 'c' no final
    "cccacbc"   # Muitos 'c' no início, depois mix
]

# Extrair representações do encoder
model.eval()
encodings = []
with torch.no_grad():
    for seq in sequences:
        src = pad(encode(seq)).unsqueeze(0).to(device, dtype=torch.long)
        h = model.encoder(src)  # [1, 1, hidden_size]
        encodings.append(h.squeeze().cpu().numpy())

encodings = np.array(encodings)  # [4, hidden_size]

# Aplicar PCA para reduzir para 2 dimensões
pca = PCA(n_components=2)
encodings_2d = pca.fit_transform(encodings)

# Visualização
plt.figure(figsize=(12, 5))

# Subplot 1: Gráfico PCA
plt.subplot(1, 2, 1)
colors = ['red', 'blue', 'green', 'orange']
for i, (seq, color) in enumerate(zip(sequences, colors)):
    plt.scatter(encodings_2d[i, 0], encodings_2d[i, 1],
               c=color, s=200, alpha=0.6, edgecolors='black', linewidth=2)
    plt.annotate(seq, (encodings_2d[i, 0], encodings_2d[i, 1]),
                fontsize=12, ha='center', va='bottom', fontweight='bold')

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variância)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variância)', fontsize=12)
plt.title('Representações do Encoder em 2D (PCA)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Subplot 2: Análise de similaridade
plt.subplot(1, 2, 2)
similarities = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        # Distância euclidiana entre representações
        dist = np.linalg.norm(encodings[i] - encodings[j])
        similarities[i, j] = dist

im = plt.imshow(similarities, cmap='YlOrRd', aspect='auto')
plt.colorbar(im, label='Distância Euclidiana')
plt.xticks(range(4), sequences, rotation=45)
plt.yticks(range(4), sequences)
plt.title('Matriz de Distâncias entre Sequências', fontsize=14, fontweight='bold')

for i in range(4):
    for j in range(4):
        text = plt.text(j, i, f'{similarities[i, j]:.2f}',
                       ha="center", va="center", color="black", fontsize=10)

plt.tight_layout()
plt.savefig('pca_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

# Análise textual
print("\n📊 ANÁLISE DOS RESULTADOS:")
print("\n1. Comparação de sequências SIMILARES:")
print(f"   - '{sequences[0]}' vs '{sequences[1]}':")
print(f"     Distância: {similarities[0, 1]:.3f}")
print(f"     Análise: Ambas têm muitos 'a' e 'b', mas ordem diferente")

print(f"\n   - '{sequences[2]}' vs '{sequences[3]}':")
print(f"     Distância: {similarities[2, 3]:.3f}")
print(f"     Análise: Ambas têm muitos 'c', mas distribuição diferente")

print("\n2. Comparação de sequências DIFERENTES:")
print(f"   - '{sequences[0]}' vs '{sequences[2]}':")
print(f"     Distância: {similarities[0, 2]:.3f}")
print(f"     Análise: Composição muito diferente (a/b vs c/a)")

print(f"\n   - '{sequences[1]}' vs '{sequences[3]}':")
print(f"     Distância: {similarities[1, 3]:.3f}")
print(f"     Análise: Composição e ordem bem distintas")

print(f"\n3. Variância explicada pelo PCA:")
print(f"   - PC1: {pca.explained_variance_ratio_[0]:.1%}")
print(f"   - PC2: {pca.explained_variance_ratio_[1]:.1%}")
print(f"   - Total: {sum(pca.explained_variance_ratio_):.1%}")

print("\n💡 CONCLUSÕES:")
print("   ✓ O encoder captura padrões de composição das sequências")
print("   ✓ Sequências com caracteres similares ficam mais próximas")
print("   ✓ A ordem dos caracteres também afeta a representação")
print("   ✓ O PCA consegue separar bem as sequências no espaço 2D")